In [43]:
import pandas as pd

In [46]:
cd vl_optimizer

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


# Chiller Group

total_consumption_chiller_group = β1*number_active_chillers + β2*outdoor_temperature + β3*outdoor_humidity + β4*Chiller Set point + β5*Chiller Efficiency

In [47]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-06T21-45-14_industrial_site2_processed.csv')

In [48]:
df.columns

Index(['Time', '02 Production_Electric_Active Energy (kWh)',
       '02 Production_Electric_Active Power (kW)',
       '02 Production_Electric_Apparent Power (kVa)',
       '02 Production_Electric_Power Factor (real)',
       '03 Chiller Group_Electric_Active Energy (kWh)',
       '03 Chiller Group_Electric_Active Power (kW)',
       '03 Chiller Group_Electric_Apparent Power (kVa)',
       '03 Chiller Group_Electric_Power Factor (real)',
       '04 UTA_Electric_Active Energy (kWh)',
       '04 UTA_Electric_Active Power (kW)',
       '04 UTA_Electric_Apparent Power (kVa)',
       '04 UTA_Electric_Power Factor (real)',
       '05 Compressors_Electric_Active Energy (kWh)',
       '05 Compressors_Electric_Active Power (kW)',
       '05 Compressors_Electric_Apparent Power (kVa)',
       '05 Compressors_Electric_Power Factor (real)',
       '06 Offices_Electric_Active Energy (kWh)',
       '06 Offices_Electric_Active Power (kW)',
       '06 Offices_Electric_Apparent Power (kVa)',
       '06 

In [49]:
columns_to_include = ['Time', '03 Chiller Group_Electric_Active Energy (kWh)', 'temp', 'humidity', 'heat_index', '03 Chiller Group_Electric_Power Factor (real)']

df_for_synthesis = df[columns_to_include]

df_for_synthesis.columns

Index(['Time', '03 Chiller Group_Electric_Active Energy (kWh)', 'temp',
       'humidity', 'heat_index',
       '03 Chiller Group_Electric_Power Factor (real)'],
      dtype='object')

In [50]:
df_for_synthesis = df_for_synthesis.rename(columns={'temp': 'outdoor_temp', 'humidity': 'outdoor_humidity', '03 Chiller Group_Electric_Power Factor (real)': 'Chiller Group_Electric_Power Factor (real)(efficiency)'})

In [51]:
df_for_synthesis.head()

,Time,03 Chiller Group_Electric_Active Energy (kWh),outdoor_temp,outdoor_humidity,heat_index,Chiller Group_Electric_Power Factor (real)(efficiency)
0,2022-07-01 00:00:00,195.50,15.97,0.37,15.97,0.78375
1,2022-07-01 01:00:00,195.25,15.10,0.37,15.10,0.77975
2,2022-07-01 02:00:00,162.25,14.80,0.38,14.80,0.79450
3,2022-07-01 03:00:00,157.50,15.10,0.38,15.10,0.78075
4,2022-07-01 04:00:00,158.25,15.97,0.38,15.97,0.78475


In [52]:
df_for_synthesis.to_csv('data/target_variables/target_chiller_group.csv')

In [53]:
file_path = 'data/target_variables/target_chiller_group.csv'
data = pd.read_csv(file_path)
data.columns

Index(['Unnamed: 0', 'Time', '03 Chiller Group_Electric_Active Energy (kWh)',
       'outdoor_temp', 'outdoor_humidity', 'heat_index',
       'Chiller Group_Electric_Power Factor (real)(efficiency)'],
      dtype='object')

In [54]:
import numpy as np
import pandas as pd

# Define the function to calculate chiller settings
def calculate_chiller_settings(heat_index):
    # Calculate chiller set point
    if heat_index < 18:
        set_point = 18
    elif heat_index > 40:
        set_point = 25
    else:
        set_point = round(18 + (heat_index - 18) * (7 / (40 - 18)))
    
    # Calculate number of active chillers
    if heat_index < 18:
        active_chillers = 0
    else:
        # Simple model: the number of chillers increases with heat_index
        # This is a simplistic linear relationship for demonstration purposes
        heat_index_factor = (heat_index - 18) / (40 - 18)
        active_chillers = int(np.ceil(heat_index_factor * 10))  # Example scale factor, adjust as needed
        
    return set_point, active_chillers

# Assuming your existing DataFrame is named df and has columns 'outdoor_temp' and 'humidity'
# Apply the function to each row in the DataFrame
data[['set_point', 'num_active_chiller']] = data.apply(
    lambda row: pd.Series(calculate_chiller_settings(row['heat_index'])),
    axis=1
)

data.head()

,Unnamed: 0,Time,03 Chiller Group_Electric_Active Energy (kWh),outdoor_temp,outdoor_humidity,heat_index,Chiller Group_Electric_Power Factor (real)(efficiency),set_point,num_active_chiller
0,0,2022-07-01 00:00:00,195.50,15.97,0.37,15.97,0.78375,18,0
1,1,2022-07-01 01:00:00,195.25,15.10,0.37,15.10,0.77975,18,0
2,2,2022-07-01 02:00:00,162.25,14.80,0.38,14.80,0.79450,18,0
3,3,2022-07-01 03:00:00,157.50,15.10,0.38,15.10,0.78075,18,0
4,4,2022-07-01 04:00:00,158.25,15.97,0.38,15.97,0.78475,18,0


In [55]:


data.drop('Unnamed: 0')
# Ensure your DataFrame 'df' has the necessary columns
# Assuming df already has 'outdoor_temp', 'outdoor_humidity', 'set_point', and 'num_active_chiller' columns



KeyError: "['Unnamed: 0'] not found in axis"

In [56]:
data.to_csv('data/synthesized_data/chiller_group_synthesized.csv', index=False)